In [1]:
from model_helpers import *

import cfgrib
import xarray as xr

import pandas as pd
import numpy as np

from pyPhenology import models, utils

from tqdm import trange, tqdm

import matplotlib.pyplot as plt

from warnings import warn
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
high_cutoff_year = 2022
low_cutoff_year = 2010

In this notebook, we implement two methods for increasing the accuracy of our ripeness predictions:

1. higher space resolution. 
2. train on all years before 2022, test on 2022. 

In [4]:
# Load in high-res weather data
grib_data = cfgrib.open_datasets('../data/monthly_weather_data.grib')

core_data = grib_data[0]

In [5]:
## Load observations
formatted_plants = pd.read_csv("../data/model_training_data/all_plants_formatted.csv", index_col=0)


In [6]:
formatted_plants['rounded_lat'] = np.round(formatted_plants['latitude'], 1)
formatted_plants['rounded_lon'] = np.round(formatted_plants['lon_360'], 1)

rounded_sites = formatted_plants[['site_id', 'rounded_lat', 'rounded_lon']].drop_duplicates()

site_x_vals = xr.DataArray(rounded_sites['rounded_lat'], dims=['site'])
site_y_vals = xr.DataArray(rounded_sites['rounded_lon'], dims=['site'])

In [7]:
full_weather_data = core_data.sel(latitude=site_x_vals, longitude=site_y_vals, method='nearest').to_dataframe().dropna()

In [8]:
formatted_weather = format_weather_data(full_weather_data)

formatted_weather['latitude'] = np.round(formatted_weather['latitude'], 1)
formatted_weather['longitude'] = np.round(formatted_weather['longitude'], 1)

In [9]:
rounded_sites['coordstring'] = rounded_sites['rounded_lat'].astype(str) + rounded_sites['rounded_lon'].astype(str)
formatted_weather['coordstring'] = formatted_weather['latitude'].astype(str) + formatted_weather['longitude'].astype(str)



In [10]:
## Add Site ID to the weather data
weather_with_sites = pd.merge(formatted_weather, rounded_sites[['coordstring', 'site_id']], on='coordstring').drop('coordstring', axis=1)
## Separate into training data and testing data

# filter out current year
weather_with_sites = weather_with_sites[weather_with_sites['year'] != 2023]

weather_training = weather_with_sites[weather_with_sites['year'] < high_cutoff_year]
weather_test = weather_with_sites[weather_with_sites['year'] >= high_cutoff_year]



In [11]:
species_list = formatted_plants['formatted_sci_name'].unique()

In [12]:
formatted_plants.drop('species', axis=1, inplace=True)

In [13]:
weather_sites = weather_with_sites['site_id'].unique()

filtered_plants = formatted_plants[(formatted_plants['site_id'].isin(weather_sites)) & 
                                   (formatted_plants['year'] != 2023) &
                                   (formatted_plants['latitude'] > 0)]



In [14]:
def make_test_df(train_df):
    #print(train_df)
    species_sites = train_df['site_id'].unique()
        
    #print(species_sites)
    
    site_ripenesses = []

    for site in species_sites:
        site_df = train_df[train_df['site_id'] == site]

        site_ripenesses.append({
            'site_id': site,
            'doy': np.mean(site_df['doy'])
        })

    species_test_df = pd.DataFrame(site_ripenesses)
    species_test_df['year'] = high_cutoff_year
    
    return species_test_df

In [15]:
## Train models

species_prediction_dict = {}

for s in tqdm(species_list):
    print(s)
    species_train_df = filtered_plants.query('formatted_sci_name == "{}" and year < {}'.format(s, high_cutoff_year))
    
    if len(species_train_df) == 0:
        continue
    
    species_test_df = filtered_plants.query('formatted_sci_name == "{}" and year >= {}'.format(s, high_cutoff_year))
    
    if len(species_test_df) == 0:
        # make predictions and compare to the mean ripeness day at each site
        species_test_df = make_test_df(species_train_df)
    
    if len(species_test_df) == 0:
        print("No test data for {}".format(s))
        #print(species_test_df)
        
    predictions = train_ripeness_small(species_train_df, weather_training,
                        species_test_df, weather_test)
    
    species_prediction_dict[s] = predictions

  0%|                                                                                                                                                                              | 0/97 [00:00<?, ?it/s]

Rubus
running model ThermalTime
making predictions for model ThermalTime


  1%|█▋                                                                                                                                                                    | 1/97 [00:04<06:56,  4.34s/it]

model ThermalTime got a MAE of 28.6
model ThermalTime got an RMSE of 49.24970389081881
model ThermalTime's median error is: 3.0
Ripeness Day: 187.13333333333333
Rubus occidentalis
running model ThermalTime
making predictions for model ThermalTime


  2%|███▍                                                                                                                                                                  | 2/97 [00:07<05:31,  3.49s/it]

...

making predictions for model ThermalTime
model ThermalTime got a MAE of 19.625
model ThermalTime got an RMSE of 24.902309129877896
model ThermalTime's median error is: -14.5
Ripeness Day: 245.0
Prunus nigra
running model ThermalTime
making predictions for model ThermalTime


 40%|██████████████████████████████████████████████████████████████████▎                                                                                                  | 39/97 [01:59<03:19,  3.45s/it]

model ThermalTime got a MAE of 55.0
model ThermalTime got an RMSE of 55.0
model ThermalTime's median error is: -55.0
Ripeness Day: 275.0
Prunus cerasifera
running model ThermalTime


 40%|██████████████████████████████████████████████████████████████████▎                                                                                                  | 39/97 [02:00<02:59,  3.09s/it]


KeyboardInterrupt: 

In [ ]:
## Train models

genus_prediction_dict = {}

for s in tqdm(filtered_plants['genus'].unique()):
    print(s)
    species_train_df = filtered_plants.query('genus == "{}" and year < {}'.format(s, high_cutoff_year))
    
    if len(species_train_df) == 0:
        continue
    
    species_test_df = filtered_plants.query('genus == "{}" and year >= {}'.format(s, high_cutoff_year))
    
    if len(species_test_df) == 0:
        # make predictions and compare to the mean ripeness day at each site
        species_test_df = make_test_df(species_train_df)
    
    if len(species_test_df) == 0:
        print("No test data for {}".format(s))
        #print(species_test_df)
        
    predictions = train_ripeness_small(species_train_df, weather_training,
                        species_test_df, weather_test)
    
    genus_prediction_dict[s] = predictions

In [23]:
pd.DataFrame(filtered_plants.groupby('formatted_sci_name').size()).rename(columns={0:'n'}).query('n > 10')

,n
formatted_sci_name,
Amelanchier,318
Amelanchier alnifolia,189
Amelanchier arborea,127
Amelanchier canadensis,199
Amelanchier grandiflora,251
Amelanchier laevis,86
Citrus,571
Cornus mas,45
Diospyros,63


In [25]:
pd.DataFrame(filtered_plants.groupby('genus').size()).rename(columns={0:'n'}).query('n > 1')

,n
genus,
Amelanchier,1176
Citrus,588
Cornus,45
Diospyros,445
Eriobotrya,223
Ficus,229
Juglans,349
Malus,13553
Morus,1090


## Model Validation

In [ ]:
#full_predictions = pd.concat(list(species_prediction_dict.values()))
genus_predictions = pd.concat(list(species_prediction_dict.values()))

In [ ]:
full_predictions

In [ ]:
full_predictions['abs_error'] = np.abs(full_predictions['doy'] - full_predictions['flowering_day'])
full_predictions['square_error'] = full_predictions['abs_error'] ** 2

In [ ]:
full_predictions.sort_values('abs_error', ascending=False)[0:50]

In [ ]:
np.mean(full_predictions['abs_error'])

In [ ]:
np.median(full_predictions['abs_error'])

In [ ]:
len(full_predictions.query('abs_error < 30.6')) / len(full_predictions)

In [ ]:
np.std(full_predictions['abs_error'])

In [ ]:
len(full_predictions.query('abs_error < 43.09')) / len(full_predictions)

In [ ]:
plt.scatter(full_predictions['doy'], full_predictions['flowering_day'])

Idea for corrections: take the lower error between the base error and the year-transformed error.

NEW TRAINING PARADIGM:
Specific data if possible, more generalized data if not.
Ex. use European high-time-res data if possible, otherwise use monthly data. 
Use species if possible, otherwise use genus. 

High error is driven by southern hemisphere and "date wrapping". 

Removing southern hemisphere data and dates that are either very early in the year or very late works a little better.

Ok, median error seems to be a better error metric. the old (coarse) method had a Median Absolute Error of 25, while this one has one of 14. 

After doing some more analysis, 71% of observations fall under the mean (30 days) and 81% fall under the standard deviation (43 days). 

**Trying with no northern hemisphere / Date Wrapping**

In [ ]:
unwrapped_northern_preds = full_predictions.query('latitude > 0 and doy > 60')

In [ ]:
unwrapped_northern_preds.sort_values('abs_error', ascending=False)[0:50]

In [ ]:
plt.hist(full_predictions['abs_error'], bins=50)

In [ ]:
np.median(full_predictions['abs_error'])

In [ ]:
print(unwrapped_northern_preds)

print(rmse(unwrapped_northern_preds['doy'], unwrapped_northern_preds['flowering_day']))
print(mae(unwrapped_northern_preds['doy'], unwrapped_northern_preds['flowering_day']))

In [ ]:
def rmse(y1, y2):
    return np.sqrt(np.mean((y1 - y2) ** 2))

def mae(y1, y2):
    return np.mean(np.abs(y1 - y2))

In [ ]:
small_preds = full_predictions.dropna()

In [ ]:
print(rmse(full_predictions['doy'], full_predictions['flowering_day']))
print(rmse(small_preds['doy'], small_preds['flowering_day']))
print(mae(full_predictions['doy'], full_predictions['flowering_day']))
print(mae(small_preds['doy'], small_preds['flowering_day']))

Old MAE: 31.9; new MAE: 30.6.